# CFG options compared

This notebook compares three options to generate text from context-free grammars in Python:
- Tracery
- NLTK
- From scratch

Each option is demonstrated with two examples from [Gary Bernhardt's screencasts](https://www.destroyallsoftware.com/screencasts/catalog), "Computation Explained Briskly, for Programmers."

- From [Recognizing Simple Languages](https://www.destroyallsoftware.com/screencasts/catalog/recognizing-simple-languages)

    ```
    r"""
        EXPR = NUM
        EXPR = NUM OP EXPR
        NUM = '1'
        NUM = '2'
        NUM = '3'
        OP = '+'
        OP = '*'
    """
    ```

- From [Recognizing Programming Languages](https://www.destroyallsoftware.com/screencasts/catalog/recognizing-programming-languages)

    ```
    S = ''
    S = '(' S ')'
    ```

# Tracery

- Installation: `pip install tracery`
- [Examples](https://github.com/aparrish/rwet/blob/master/tracery-and-python.ipynb)

In [1]:
import tracery

rules = {
    'EXPR': ['#NUM#', '#NUM# #OP# #EXPR#'], 
    'NUM':  ['1', '2', '3'], 
    'OP':   ['+', '*']
}

grammar = tracery.Grammar(rules)

for _ in range(5):
    print(grammar.flatten('#EXPR#'))

1
2 * 1
1
3 + 3
1


In [2]:
rules = {
    'S': ['', '(#S#)']
}

grammar = tracery.Grammar(rules)

for _ in range(5):
    print(grammar.flatten('#S#'))

(())

(())
()
()


# NLTK

- Installation: `pip install nltk`
- [Examples](http://www.nltk.org/howto/generate.html)
- [*Natural Language Processing with Python*, Chapter 8,](http://www.nltk.org/book/ch08.html) Section 3: Context Free Grammar

In [3]:
from nltk import CFG
from nltk.parse.generate import generate

rules = """
    EXPR -> NUM
    EXPR -> NUM OP EXPR
    NUM -> '1'
    NUM -> '2'
    NUM -> '3'
    OP -> '+'
    OP -> '*'
"""

grammar = CFG.fromstring(rules)

# First 5, not random
for expr in generate(grammar, n=5):
    print(' '.join(expr))

1
2
3
1 + 1
1 + 2


In [4]:
rules = """
    S -> 
    S -> '(' S ')'
"""

grammar = CFG.fromstring(rules)

# First 5, not random
for S in generate(grammar, n=5):
    print(''.join(S))


()
(())
((()))
(((())))


# From Scratch

### With rules as a dictionary
- Keys are strings. Each string has one symbol.
- Values are lists of one or more strings. Each string has zero or more symbols separated by whitespace.

In [5]:
rules = {
    'EXPR': ['NUM', 'NUM OP EXPR'], 
    'NUM':  ['1', '2', '3'], 
    'OP':   ['+', '*']
}

In [6]:
import random

def generate(text, rules):
    tokenized = [token for token in text.split(' ') if token]
    
    transformed = [random.choice(rules[symbol]) if symbol in rules else symbol 
                   for symbol in tokenized]
    
    text = ' '.join(transformed)
    return text if transformed == tokenized else generate(text, rules)

In [7]:
for _ in range(5):
    print(generate('EXPR', rules))

1 * 1 + 3 * 2
3
3 * 1 + 1 + 3 + 3 + 1 + 3 + 2
1
3 * 3 * 3 + 2 + 1 * 2


In [8]:
rules = {
    'S': ['', '( S )']
}

for _ in range(5):
    print(generate('S', rules))


( ( ) )

( )
( )


### With rules as a multi-line string
- Lines contain whitespace, or rules with two sides separated by `=`
- Left sides have one symbol
- Right sides have zero or more symbols separated by whitespace

In [9]:
from collections import defaultdict

def make_rules(raw):
    rules = defaultdict(list)
    lines = [line for line in raw.split('\n') if line.strip()]
    for line in lines:
        left, right = [side.strip() for side in line.split('=')]
        rules[left].append(right)
    return rules

In [10]:
rules = make_rules("""
    
    EXPR = NUM
    EXPR = NUM OP EXPR
    NUM = 1
    NUM = 2
    NUM = 3
    OP = +
    OP = *
    
    """)

for _ in range(5):
    print(generate('EXPR', rules))

1 * 1
1
2 * 1 + 2
3 * 2 * 1
3


In [11]:
rules = make_rules("""
    
    S = ( S )
    S =
    
    """)

for _ in range(5):
    print(generate('S', rules))

( )

( )
( ( ( ) ) )

